---
**Script for Image and Video Colorization**

---

**David Norman Díaz Estrada**

davidnd@stud.ntnu.no


This script contains 3 main sections:    
**1) Load Functions for Video Clorization Framework.**

**2) Video Colorization Example.**

**3) Image Colorization Example.**

In order to run sections 2 or 3, ensure you have ran section 1 first. In sections 2 and 3 you can select any of the 5 trained models and run it for either image or video colorization.



Check current GPU:

In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Fri Nov 19 02:16:46 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   71C    P8    33W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

Check Available RAM:

In [2]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 13.6 gigabytes of available RAM

Not using a high-RAM runtime


Mount gdrive:

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


---
**SECTION 1. Load Functions for Video Colorization Framework**

---

Import required Libraries for the networks:

In [8]:
import tensorflow as tf
print("Tensorflow version: ",tf.__version__)

from keras.layers import Conv2D, UpSampling2D, Input
from keras.models import Sequential, Model
from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from skimage.color import rgb2lab, lab2rgb, gray2rgb
from skimage.transform import resize
from skimage.io import imsave

import numpy as np
print("numpy version: ",np.__version__)
import keras
print("keras version: ",keras.__version__)

import os

Tensorflow version:  2.7.0
numpy version:  1.19.5
keras version:  2.7.0


Function to load basemodel (Backbone) and trained model:

In [7]:
def loadcolorizer(selectedModel,M1_path,M2_path,M3_path,M4_path,M5_path):


  if selectedModel>=1 and selectedModel <=5: #Ensure user selects valid model

    #----------------------------------------------------------------------------------
    if selectedModel>=1 and selectedModel <=4: #Models 1,2,3,4 use VGG16 as the backbone
    #----------------------------------------------------------------------------------
      print("Loading backbone for model: M",selectedModel)
      from keras.applications.vgg16 import VGG16
      vggmodel = VGG16()
      print("VGG16 loaded")

      #----------------------------------------------------------------------------------
      if selectedModel==1: #Model 1 uses VGG16 up to the 22 layer
      #----------------------------------------------------------------------------------
        #Create a basemodel  without the last layers of original VGG16 architecture:
        basemodel = Sequential() 
        for i, layer in enumerate(vggmodel.layers):
            if i<22:          #Only up to 22th layer of size (None, 4096), (it's the last layer before dense layer of predictions)
              basemodel.add(layer)
        # show a summary of the base model
        print("[INFO] summary for base model...")
        basemodel.summary()

        #load the trained model:
        print("-"*40)
        print("Loading M1 trained model")
        print("-"*40)
        trained_Model_Path=M1_path
      #----------------------------------------------------------------------------------
      else: #Model 2,3,4 use VGG16 up to the 19 layer
      #----------------------------------------------------------------------------------
        #Create a basemodel  without the last layers of original VGG16 architecture:
        basemodel = Sequential() 
        for i, layer in enumerate(vggmodel.layers):
            if i<19:          #Only up to 19th layer to include feature extraction only
              basemodel.add(layer)
        # show a summary of the base model
        print("[INFO] summary for base model...")
        basemodel.summary()

        #load the trained model:
        print("-"*40)

        if selectedModel==2:
          print("Loading M2 trained model")
          trained_Model_Path=M2_path
        if selectedModel==3:
          print("Loading M3 trained model")
          trained_Model_Path=M3_path
        if selectedModel==4:
          print("Loading M4 trained model")
          trained_Model_Path=M4_path

        print("-"*40)
    #----------------------------------------------------------------------------------
    elif selectedModel==5: #model 5 uses resnet50 as the backbone
    #----------------------------------------------------------------------------------
      print("Loading backbone for model: M5")
      from tensorflow.keras.applications import ResNet50
      print("ResNet50 loaded")

      basemodel = ResNet50(weights="imagenet", include_top=False,
      input_tensor=Input(shape=(224, 224, 3)))
      # show a summary of the base model
      print("[INFO] summary for base model...")
      print(basemodel.summary())

      print("-"*40)
      print("Loading M5 trained model")
      print("-"*40)
      trained_Model_Path=M5_path

    #--------------------------------------------------------------------------------
    # Load the trained model
    #--------------------------------------------------------------------------------
    print("[INFO] summary for trained model...")
    model = tf.keras.models.load_model(trained_Model_Path,
                                      custom_objects=None,
                                      compile=True)
    model.summary()

    return [model,basemodel]
  else:
    print("Select a valid model: 1,2,3,4,5 ")



Import libraries required for video processing:

In [6]:
import cv2
print("OpenCV version: ",cv2.__version__)
import numpy as np
import glob
from time import time
import os

OpenCV version:  4.1.2


Function to extract all frames from video and save in a given folder:


In [7]:
def extractFrames(original_video_path):
  # this function extracts all frames from a given video (maximum 1000 frames)
  # it saves frames in following order: (001,002,003... 010,020,100,200...1000)

  print("Extracting frames from video...")
  video = cv2.VideoCapture(original_video_path)
  success,image = video.read()

  #get the number of frames per second of original video(FPS):
  fps = int(video.get(cv2.CAP_PROP_FPS))
  print("Frames per Second: "+str(fps))
  #get total number of frames in video:
  totalFrames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
  print("Total frames: "+str(totalFrames))
  #compute time lenght of video:
  videoTimeSeconds=totalFrames/fps
  print("Video Lenght: "+str(videoTimeSeconds)+" seconds")

  if totalFrames<=1000:
    count = 1
    while success:
      if count<10:  frameNmbr='00'+str(count)
      if count>=10 and count<100: frameNmbr='0'+str(count)
      if count>=100 and count<=1000: frameNmbr=str(count)  
      #print(frameNmbr) 

      #cv2.imwrite("/content/videoFrames2/frame%s.jpg" % frameNmbr, image)     # save frame as JPEG file
      framePath=original_frames_path+"/frame"+frameNmbr+".jpg" 
      cv2.imwrite(framePath, image)     # save frame as JPEG file 

      success,image = video.read() #image is type: <class 'numpy.ndarray'> 
      count += 1
  else:
    print("Video has more than 1000 frames, not suitable for demo")

  return fps

Function to colorize (predict) a folder with images: 

In [6]:
def colorizeFrames(original_frames_path,colorized_frames_path,model,basemodel,selectedModel):
  print("Colorizing frames...")
  # For this cell the I took some inspiration from :
  # https://github.com/bnsreenu/python_for_microscopists/blob/master/092-autoencoder_colorize_transfer_learning_VGG16_V0.1.py

  ############################################
  #Predicting using trained colorization model.
  ############################################

  #contador=1

  files = os.listdir(original_frames_path)
  for idx, file in enumerate(files):
    #try:
      test = img_to_array(load_img(original_frames_path+"/"+file))
      #print("Type: ",type(test), " Shape: ",test.shape)
      #print(test[:,:,0])
      #print("-"*20)
      #print(test[:,:,1])
      #print("-"*20)
      #print(test[:,:,2])
      #print("Type: ",type(test[1,1,0]), " Shape: ",test[:,:,0].shape)


      test = resize(test, (224,224), anti_aliasing=True)
      test*= 1.0/255
      lab = rgb2lab(test)
      l = lab[:,:,0]
      L = gray2rgb(l)
      L = L.reshape((1,224,224,3))
      #print(L.shape)
      basemodel_features = basemodel.predict(L)
      
      if selectedModel==1:
        ab = model.predict([basemodel_features,L])
      if (selectedModel in range(2,6)):
        ab = model.predict(basemodel_features)
      
      
      #print(ab.shape)
      ab = ab*128
      cur = np.zeros((224, 224, 3))
      cur[:,:,0] = l
      cur[:,:,1:] = ab

      #imsave('/content/videoFramesColorized2/'+file, lab2rgb(cur))
      #imsave('/content/testColorized/'+file, lab2rgb(cur))
      imsave(colorized_frames_path+'/'+"m"+str(selectedModel)+"_"+file, lab2rgb(cur))
    #except:
      #print("************An error ocurred")

    #if contador==1: break

Function to build video from images in folder:


In [9]:
def buildVideo(fps,colorized_frames_path,selectedModel):
  t1 = time()
  frameSize = (224, 224)
  framesPerSecond=fps
  video_file='colorizedvideo_model'+str(selectedModel)+"_"+str(framesPerSecond)+'fps.mp4'
  out = cv2.VideoWriter(video_file, cv2.VideoWriter_fourcc(*'mp4v'), framesPerSecond, frameSize)

  #################################################
  #Build the video stacking all frames:
  #################################################
  #Access all files in path:
  #files = glob.glob(r'/content/videoFramesColorized2/*.jpg')
  files = glob.glob(colorized_frames_path+'/*.jpg')
  #Sort the files by alphanumeric order:
  files.sort()


  #Now iterate over each image and add it to the video:
  for myfile in files:
      #print('\n'+myfile)
      img = cv2.imread(myfile)
      #print("type: ",type(img),"shape: ",img.shape, " dataType: ",type(img[1,1,0]))
      out.write(img)


  out.release()#finish video writting
  print("Finished writting video")
  t2 = time()
  elapsed = t2 - t1
  print('Elapsed time is %f seconds.' % elapsed)

---
**SECTION 2: Video Colorization Example (step by step):**

---

The Video colorization framework is summarized in the following steps:

1) Define path for trained models, input video, and specify the desired colorization model to use (model 1,2,3,4 or 5).

2) Load the trained model for colorization.

3) Extract frames from input video.

4) Colorize frames.

5) Build video from colorized frames.


**Step 1):**

Define path for trained models, input video, and specify the desired colorization model to use (model 1,2,3,4 or 5)

In [55]:
################################################
# Set the paths for each of the trained models: 
################################################
M1_path='/content/gdrive/MyDrive/COSI/Semestre3/VideoProcessing/trainedModels/M1_colorizer_fusionModel_train61-36_val62-53_100epochs_batch8.h5'
M2_path='/content/gdrive/MyDrive/COSI/Semestre3/VideoProcessing/trainedModels/M2_colorizer_train84-92_val63-20__1000epochs_batch16_1000imgs.h5'
M3_path='/content/gdrive/MyDrive/COSI/Semestre3/VideoProcessing/trainedModels/M3_colorizer_train83-73_val65-57__1000epochs_batch16_2000imgs.h5'
M4_path='/content/gdrive/MyDrive/COSI/Semestre3/VideoProcessing/trainedModels/M4_colorizer_train81-37_val63-58__1000epochs_batch16_8000imgs.h5'
M5_path='/content/gdrive/MyDrive/COSI/Semestre3/VideoProcessing/trainedModels/M5_colorizer_train81-28_val62-92__1000epochs_batch16_8000imgs_resnet50.h5'

################################################
# Set paths for video:
################################################

#Set path for original video:
original_video_path="/content/gdrive/MyDrive/COSI/Semestre3/VideoProcessing/testVideos/interlaken1.mp4"
#Set path to save extracted frames from original video:
original_frames_path="/content/frames_original_interlaken1"#"/content/frames_original"
#Set path to save colorized frames:
colorized_frames_path="/content/frames_colorized_interlaken1_m5"#"/content/frames_colorized"

################################################
# Select Colorization Model:
################################################

#Indicate which model to use for colorization: 1,2,3,4,5 , which correspond to M1,M2,M3,M4,M5, respectively:
selectedModel=5

**Step 2): Load the trained model for colorization.**

The following function returns the "model" which corresponds to the trained model, and the "basemodel" which corresponds to the pre-trained model for feature extraction (either VGG16 or ResNet50):

In [56]:
model,basemodel=loadcolorizer(selectedModel,M1_path,M2_path,M3_path,M4_path,M5_path)

Loading backbone for model: M5
ResNet50 loaded
[INFO] summary for base model...
Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_7 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_7[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
           

**Step 3): Extract frames from the input video.**

This function will return the FPS (frames per second) of the original video, to ensure the colorized video will be built with the same FPS.

In [57]:
fps=extractFrames(original_video_path)

Extracting frames from video...
Frames per Second: 30
Total frames: 455
Video Lenght: 15.166666666666666 seconds


**Step 4): Colorize Frames.**

In [52]:
colorizeFrames(original_frames_path,colorized_frames_path,model,basemodel,selectedModel)

Colorizing frames...


/usr/local/lib/python3.7/dist-packages/skimage/color/colorconv.py:1109: UserWarning: Color data out of range: Z < 0 in 245 pixels
  return xyz2rgb(lab2xyz(lab, illuminant, observer))
/usr/local/lib/python3.7/dist-packages/skimage/color/colorconv.py:1109: UserWarning: Color data out of range: Z < 0 in 3 pixels
  return xyz2rgb(lab2xyz(lab, illuminant, observer))
/usr/local/lib/python3.7/dist-packages/skimage/color/colorconv.py:1109: UserWarning: Color data out of range: Z < 0 in 83 pixels
  return xyz2rgb(lab2xyz(lab, illuminant, observer))
/usr/local/lib/python3.7/dist-packages/skimage/color/colorconv.py:1109: UserWarning: Color data out of range: Z < 0 in 20 pixels
  return xyz2rgb(lab2xyz(lab, illuminant, observer))
/usr/local/lib/python3.7/dist-packages/skimage/color/colorconv.py:1109: UserWarning: Color data out of range: Z < 0 in 1 pixels
  return xyz2rgb(lab2xyz(lab, illuminant, observer))
/usr/local/lib/python3.7/dist-packages/skimage/color/colorconv.py:1109: UserWarning: Color 

**Step 5): Build video from colorized frames.**

In [53]:
buildVideo(fps,colorized_frames_path,selectedModel)

Finished writting video
Elapsed time is 0.421726 seconds.


---
**SECTION 3: Image Colorization Example**

---

In [1]:
############################################################
# Set the paths for each of the trained colorization models: 
############################################################
M1_path='/content/gdrive/MyDrive/COSI/Semestre3/VideoProcessing/trainedModels/M1_colorizer_fusionModel_train61-36_val62-53_100epochs_batch8.h5'
M2_path='/content/gdrive/MyDrive/COSI/Semestre3/VideoProcessing/trainedModels/M2_colorizer_train84-92_val63-20__1000epochs_batch16_1000imgs.h5'
M3_path='/content/gdrive/MyDrive/COSI/Semestre3/VideoProcessing/trainedModels/M3_colorizer_train83-73_val65-57__1000epochs_batch16_2000imgs.h5'
M4_path='/content/gdrive/MyDrive/COSI/Semestre3/VideoProcessing/trainedModels/M4_colorizer_train81-37_val63-58__1000epochs_batch16_8000imgs.h5'
M5_path='/content/gdrive/MyDrive/COSI/Semestre3/VideoProcessing/trainedModels/M5_colorizer_train81-28_val62-92__1000epochs_batch16_8000imgs_resnet50.h5'


In [4]:
#Load the testing images from the Places365 dataset:
!unzip '/content/gdrive/My Drive/COSI/Semestre3/VideoProcessing/places365_11k/test.zip'

Archive:  /content/gdrive/My Drive/COSI/Semestre3/VideoProcessing/places365_11k/test.zip
   creating: test/
  inflating: test/Places365_val_00010248.jpg  
  inflating: test/Places365_val_00010249.jpg  
  inflating: test/Places365_val_00010250.jpg  
  inflating: test/Places365_val_00010251.jpg  
  inflating: test/Places365_val_00010252.jpg  
  inflating: test/Places365_val_00010253.jpg  
  inflating: test/Places365_val_00010254.jpg  
  inflating: test/Places365_val_00010255.jpg  
  inflating: test/Places365_val_00010256.jpg  
  inflating: test/Places365_val_00010257.jpg  
  inflating: test/Places365_val_00010258.jpg  
  inflating: test/Places365_val_00010259.jpg  
  inflating: test/Places365_val_00010260.jpg  
  inflating: test/Places365_val_00010261.jpg  
  inflating: test/Places365_val_00010262.jpg  
  inflating: test/Places365_val_00010263.jpg  
  inflating: test/Places365_val_00010264.jpg  
  inflating: test/Places365_val_00010265.jpg  
  inflating: test/Places365_val_00010266.jpg  

In [9]:
#Colorize images:

selectedModel=4 #select which model to use (1,2,3,4 or 5)
images_path='/content/test' #define path for the test images folder
colorized_images_path='/content/test_colorized' #define path where the colorized images will be saved

model,basemodel=loadcolorizer(selectedModel,M1_path,M2_path,M3_path,M4_path,M5_path) #load the colorizer model
colorizeFrames(images_path,colorized_images_path,model,basemodel,selectedModel) #colorize the images in the specified folder


Loading backbone for model: M 4
553476096/553467096 [==============================] - 7s 0us/step
VGG16 loaded
[INFO] summary for base model...
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56,

/usr/local/lib/python3.7/dist-packages/skimage/color/colorconv.py:1109: UserWarning: Color data out of range: Z < 0 in 3 pixels
  return xyz2rgb(lab2xyz(lab, illuminant, observer))
/usr/local/lib/python3.7/dist-packages/skimage/color/colorconv.py:1109: UserWarning: Color data out of range: Z < 0 in 40 pixels
  return xyz2rgb(lab2xyz(lab, illuminant, observer))
/usr/local/lib/python3.7/dist-packages/skimage/color/colorconv.py:1109: UserWarning: Color data out of range: Z < 0 in 42 pixels
  return xyz2rgb(lab2xyz(lab, illuminant, observer))
/usr/local/lib/python3.7/dist-packages/skimage/color/colorconv.py:1109: UserWarning: Color data out of range: Z < 0 in 2 pixels
  return xyz2rgb(lab2xyz(lab, illuminant, observer))
/usr/local/lib/python3.7/dist-packages/skimage/color/colorconv.py:1109: UserWarning: Color data out of range: Z < 0 in 1 pixels
  return xyz2rgb(lab2xyz(lab, illuminant, observer))
/usr/local/lib/python3.7/dist-packages/skimage/color/colorconv.py:1109: UserWarning: Color da

In [10]:
#Create a zip file that contains the colorized images, so that you can download all of them:
import zipfile
!zip -r /content/test_colorized.zip /content/test_colorized

  adding: content/test_colorized/ (stored 0%)
  adding: content/test_colorized/m4_Places365_val_00010939.jpg (deflated 1%)
  adding: content/test_colorized/m4_Places365_val_00010413.jpg (deflated 2%)
  adding: content/test_colorized/m4_Places365_val_00011163.jpg (deflated 2%)
  adding: content/test_colorized/m4_Places365_val_00010951.jpg (deflated 1%)
  adding: content/test_colorized/m4_Places365_val_00010991.jpg (deflated 1%)
  adding: content/test_colorized/m4_Places365_val_00010812.jpg (deflated 1%)
  adding: content/test_colorized/m4_Places365_val_00010502.jpg (deflated 1%)
  adding: content/test_colorized/m4_Places365_val_00010631.jpg (deflated 1%)
  adding: content/test_colorized/m4_Places365_val_00010778.jpg (deflated 1%)
  adding: content/test_colorized/m4_Places365_val_00010662.jpg (deflated 1%)
  adding: content/test_colorized/m4_Places365_val_00011075.jpg (deflated 1%)
  adding: content/test_colorized/m4_Places365_val_00010368.jpg (deflated 1%)
  adding: content/test_coloriz